## Computing CLEVER metric with CNN model from Keras trained with CIFAR10 dataset

In [11]:
# import neccessary files
from __future__ import absolute_import, division, print_function, unicode_literals

from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Activation, Dropout
import numpy as np
import tensorflow as tf

# art library
from art import metrics
from art.attacks import DeepFool
from art.classifiers import KerasClassifier
from art.utils import load_dataset


## Preparing dataset and environment
In this file, we will use the CIFAR10 dataset.
We will disable tensorflow 2.0's eager evaluation as ART has not been updated to support eager evaluation.

In [5]:
# disable tensorflow 2.0 eager evaluation as it is not yet supported
tf.compat.v1.disable_eager_execution()
# Read CIFAR10 dataset
(x_train, y_train), (x_test, y_test), min_, max_ = load_dataset(str('cifar10'))
x_train, y_train = x_train[:5000], y_train[:5000]
x_test, y_test = x_test[:500], y_test[:500]
im_shape = x_train[0].shape

## Creating and preparing the model
For this example, we will use Keras Convolutional Neural Network model (CNN) without any adversarial defense training.

In [6]:
# Create Keras convolutional neural network - basic architecture from Keras examples
# Source here: https://github.com/keras-team/keras/blob/master/examples/cifar10_cnn.py
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


## Wrapping the classifier
This creates a classifier wrapper for training of model

In [7]:
# Create classifier wrapper
classifier = KerasClassifier(model=model, clip_values=(min_, max_))
classifier.fit(x_train, y_train, nb_epochs=10, batch_size=128)

Epoch 1/10
40/39 [==============================] - 3s 73ms/step - loss: 2.1371 - accuracy: 0.1824
Epoch 2/10
40/39 [==============================] - 3s 71ms/step - loss: 1.8413 - accuracy: 0.3318
Epoch 3/10
40/39 [==============================] - 3s 71ms/step - loss: 1.6529 - accuracy: 0.3994
Epoch 4/10
40/39 [==============================] - 3s 71ms/step - loss: 1.5167 - accuracy: 0.4414
Epoch 5/10
40/39 [==============================] - 3s 71ms/step - loss: 1.4400 - accuracy: 0.4678
Epoch 6/10
40/39 [==============================] - 3s 71ms/step - loss: 1.3579 - accuracy: 0.5076
Epoch 7/10
40/39 [==============================] - 3s 71ms/step - loss: 1.3506 - accuracy: 0.5193
Epoch 8/10
40/39 [==============================] - 3s 71ms/step - loss: 1.2633 - accuracy: 0.5436
Epoch 9/10
40/39 [==============================] - 3s 71ms/step - loss: 1.1854 - accuracy: 0.5713
Epoch 10/10
40/39 [==============================] - 3s 72ms/step - loss: 1.1170 - accuracy: 0.5943


## Computing of CLEVER score
```metrics.clever_u``` is used to evaluate CLEVER score for untargetted attacks.
```metrics.clever_t``` is used to evaluate CLEVER score for targetted attacks.
#### Usage
##### official documentation: <a href="https://adversarial-robustness-toolbox.readthedocs.io/en/latest/modules/metrics.html">click here</a>
```metrics.clever_u(classifier, x, nb_batches, batch_size, radius, norm, c_init=1, pool_factor=10)```

<ul>
    <li><b>classifier</b> (classifier) - classifier object we wrapped above</li>
    <li><b>x</b> (np.ndarray) - input sample (typically use x_test)</li>
    <li><b>nb_batches</b> (int) - Number of repetitions to estimate CLEVER</li>
    <li><b>batch_size</b> (int) - Number of random examples to sample per batch</li>
    <li><b>radius</b> (float) - ball of radius of the maximum perturbation</li>
    <li><b>norm</b> (int) - norm of gradient x (current support by ART: 1,2,np.inf</li>
    <li><b>c_init</b> (float) – initialization of Weibull distribution (default=1)</li>
    <li><b>pool_factor</b> (int) – The factor to create a pool of random samples with size pool_factor x n_s (default=10)</li>
</ul>


In [18]:
#using CLEVER score with first 9 test samples
scores = []
for i in range(9):
    scores.append(metrics.clever_u(classifier,x_test[i],10,10,50,1))

avg_score = sum(scores)/len(scores)
print(avg_score)
print(scores)

1.4771886582481892
[0.6599764723558106, 0.42866275691033856, 0.4143441191134822, 3.9713290118402265, 0.04674963708877289, 3.168686641416432, 1.384283950856242, 1.5588972133682568, 1.6617681212841409]
